# Task: Collect contract addresses & token_ids of all assets

**Strategy**
1. Using the assets API, start from token_id=1 and send an API request.
2. Increment token_id by 1, send a request. 
3. Store relevant fields from the response.
3. Keep incrementing until a threshold value, i.e. maximum token_id within the collection

**Relevant fields**
- token_id
- asset_contract_address
- permalink
- traits
- image_original_url
- created_date
- current_price
- last_sale
    - total_price
    - event_timestamp

### Libraries & Variables

In [14]:
from request_from_opensea import response_as_json
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_colwidth', 100)

In [15]:
# https://www.coingecko.com/en/nft/mutant-ape-yacht-club
NO_ASSETS_IN_COLLECTION = 14653

### Prototyping

In [31]:
response = response_as_json('assets', 10003)
# response = response_as_json('assets', NO_ASSETS_IN_COLLECTION)

In [34]:
if response['assets'] == []:
    print('empty')

empty


In [29]:
response['assets'][0]

{'id': 43120681,
 'token_id': '10002',
 'num_sales': 0,
 'background_color': None,
 'image_url': 'https://lh3.googleusercontent.com/pBEAE_KKNByXyRhkwmAqgESC-c-2esTce347Xb1r25n3DScRBbRhe_6Vp4f_sj4utTyWVH1nVp3L8zSeCYV5YYPS8QjClfWo9b3OuQ',
 'image_preview_url': 'https://lh3.googleusercontent.com/pBEAE_KKNByXyRhkwmAqgESC-c-2esTce347Xb1r25n3DScRBbRhe_6Vp4f_sj4utTyWVH1nVp3L8zSeCYV5YYPS8QjClfWo9b3OuQ=s250',
 'image_thumbnail_url': 'https://lh3.googleusercontent.com/pBEAE_KKNByXyRhkwmAqgESC-c-2esTce347Xb1r25n3DScRBbRhe_6Vp4f_sj4utTyWVH1nVp3L8zSeCYV5YYPS8QjClfWo9b3OuQ=s128',
 'image_original_url': 'https://ipfs.io/ipfs/QmSasKMZpHYcZJyhM6h3wdhcWzbR3X5wVFjik3Z64UPywm',
 'animation_url': None,
 'animation_original_url': None,
 'name': None,
 'description': None,
 'external_link': None,
 'asset_contract': {'address': '0x60e4d786628fea6478f785a6d7e704777c86a7c6',
  'asset_contract_type': 'non-fungible',
  'created_date': '2021-08-28T23:59:47.536067',
  'name': 'MutantApeYachtClub',
  'nft_version': 

In [6]:
int(float(response['assets'][0]['sell_orders'][0]['current_price']))/100000000

4200000000000.0

### Create Pandas Dataframe

In [46]:
columns = ["id",
           "token_id", 
           "asset_contract_address", 
           "permalink", 
           "traits", 
           "image_original_url",
           "created_date", 
           "current_price", 
           "last_sale_total_price", 
           "last_sale_event_timestamp"]
df = pd.DataFrame(columns=columns)

### Data Collection

In [47]:
for i in tqdm(range(NO_ASSETS_IN_COLLECTION-1)):
    
    response = response_as_json('assets', i)
    
    # If entry has an empty response, skip. Else, collect data.
    if response['assets']==[]:
        continue
    else:
        response_base = response['assets'][0]

        # Custom logic for current_price
        if response_base['sell_orders'] is None:
            current_price='Not on sale. Taking offers.'
        else:
            current_price=int(float(response_base['sell_orders'][0]['current_price']))/1000000000000000000

        # Custom logic for last_sale_total_price & last_sale_event_timestamp
        if response_base['last_sale'] is None:
            last_sale_total_price='Not available.'
            last_sale_event_timestamp='Not available.'
        else:
            last_sale_total_price=int(response_base['last_sale']['total_price'])/1000000000000000000
            last_sale_event_timestamp=response_base['last_sale']['event_timestamp']

        df = df.append({
            "id":response_base['id'], 
            "token_id":response_base['token_id'],
            "asset_contract_address":response_base['asset_contract']['address'], 
            "permalink":response_base['permalink'], 
            "traits":response_base['traits'], 
            "image_original_url":response_base['image_original_url'],
            "created_date":response_base['collection']['created_date'], 
            "current_price":current_price, 
            "last_sale_total_price":last_sale_total_price, 
            "last_sale_event_timestamp":last_sale_event_timestamp
        }, ignore_index=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14652/14652 [2:19:21<00:00,  1.75it/s]


In [48]:
df.head(2)

,id,token_id,asset_contract_address,permalink,traits,image_original_url,created_date,current_price,last_sale_total_price,last_sale_event_timestamp
0,42188424,0,0x60e4d786628fea6478f785a6d7e704777c86a7c6,https://opensea.io/assets/0x60e4d786628fea6478f785a6d7e704777c86a7c6/0,"[{'trait_type': 'Mouth', 'value': 'M1 Bored Unshaven', 'display_type': None, 'max_value': None, ...",https://ipfs.io/ipfs/QmURua8bNrAwX76Tp6G9t6Lospdxyt61JGy3UsXY7skfR1,2021-08-29T00:41:59.548458,Not on sale. Taking offers.,Not available.,Not available.
1,42189017,1,0x60e4d786628fea6478f785a6d7e704777c86a7c6,https://opensea.io/assets/0x60e4d786628fea6478f785a6d7e704777c86a7c6/1,"[{'trait_type': 'Mouth', 'value': 'M1 Dumbfounded', 'display_type': None, 'max_value': None, 'tr...",https://ipfs.io/ipfs/QmVFPvu8aeJHXyNVxpfftVHaKjRsbtH6dXWscJ2TukM3xi,2021-08-29T00:41:59.548458,420.0,7.0,2021-09-03T02:03:48


In [49]:
df.tail(2)

,id,token_id,asset_contract_address,permalink,traits,image_original_url,created_date,current_price,last_sale_total_price,last_sale_event_timestamp
11841,42532217,14646,0x60e4d786628fea6478f785a6d7e704777c86a7c6,https://opensea.io/assets/0x60e4d786628fea6478f785a6d7e704777c86a7c6/14646,"[{'trait_type': 'Background', 'value': 'M1 Aquamarine', 'display_type': None, 'max_value': None,...",https://ipfs.io/ipfs/QmUVXi6k83uqr8svAwdHwwu2o6S5uoGT6R9bcX5fbH5Ndi,2021-08-29T00:41:59.548458,Not on sale. Taking offers.,Not available.,Not available.
11842,42346257,14648,0x60e4d786628fea6478f785a6d7e704777c86a7c6,https://opensea.io/assets/0x60e4d786628fea6478f785a6d7e704777c86a7c6/14648,"[{'trait_type': 'Mouth', 'value': 'M1 Bored Unshaven', 'display_type': None, 'max_value': None, ...",https://ipfs.io/ipfs/QmWn8H7tDnB4DkoevNmHisnARZy5YiySBtYgohKZrfq9Jb,2021-08-29T00:41:59.548458,Not on sale. Taking offers.,9.0,2021-12-26T07:46:49


### Save as CSV

In [50]:
df.to_csv('./data/mutant-ape-collection.csv')